In [2]:
import requests
import matplotlib.pyplot as plt
import pandas as pd

1. Use the API to get all available data for the _GDP per capita, PPP (constant 2017 international $)_ indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [4]:
endpoint = 'https://api.worldbank.org/V2/country/all/indicator/NY.GDP.PCAP.PP.KD?format=json'
params = {
    'per_page': 20000,
}
response = requests.get(endpoint, params = params)

In [5]:
res_gdp = response.json()

In [6]:
res_gdp = pd.json_normalize(res_gdp[1]).drop(columns=['countryiso3code', 'unit', 'obs_status', 'indicator.id', 'indicator.value', 'decimal', 'country.id']).rename(columns={'value':'GDP_Per_Capita', 'country.value':'country'})

In [7]:
pd.DataFrame(res_gdp)

,date,GDP_Per_Capita,country
0,2023,4047.007031,Africa Eastern and Southern
1,2022,4038.638689,Africa Eastern and Southern
2,2021,3994.171654,Africa Eastern and Southern
3,2020,3919.499230,Africa Eastern and Southern
4,2019,4130.057222,Africa Eastern and Southern
...,...,...,...
17019,1964,NaN,Zimbabwe
17020,1963,NaN,Zimbabwe
17021,1962,NaN,Zimbabwe
17022,1961,NaN,Zimbabwe


2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [9]:
endpoint_le = 'https://api.worldbank.org/V2/country/all/indicator/SP.DYN.LE00.IN?format=json'
params = {
    'per_page': 20000,
}
response_le = requests.get(endpoint_le, params = params)

In [10]:
response_le.json()

[{'page': 1,
  'pages': 1,
  'per_page': 20000,
  'total': 17024,
  'sourceid': '2',
  'lastupdated': '2024-10-24'},
 [{'indicator': {'id': 'SP.DYN.LE00.IN',
    'value': 'Life expectancy at birth, total (years)'},
   'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
   'countryiso3code': 'AFE',
   'date': '2023',
   'value': None,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'SP.DYN.LE00.IN',
    'value': 'Life expectancy at birth, total (years)'},
   'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
   'countryiso3code': 'AFE',
   'date': '2022',
   'value': 62.8990306485382,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'SP.DYN.LE00.IN',
    'value': 'Life expectancy at birth, total (years)'},
   'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
   'countryiso3code': 'AFE',
   'date': '2021',
   'value': 62.4545896453863,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
 

In [11]:
res_le = response_le.json()

In [12]:
res_le = pd.json_normalize(res_le[1]).drop(columns=['countryiso3code', 'unit', 'obs_status', 'decimal', 'indicator.id', 'indicator.value', 'country.id']).rename(columns={'value':'life_expectancy','country.value':'Country'}) 

In [13]:
pd.DataFrame(res_le)

,date,life_expectancy,Country
0,2023,NaN,Africa Eastern and Southern
1,2022,62.899031,Africa Eastern and Southern
2,2021,62.454590,Africa Eastern and Southern
3,2020,63.313860,Africa Eastern and Southern
4,2019,63.755678,Africa Eastern and Southern
...,...,...,...
17019,1964,54.994000,Zimbabwe
17020,1963,54.549000,Zimbabwe
17021,1962,54.071000,Zimbabwe
17022,1961,53.619000,Zimbabwe


3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [15]:
le_gdp = pd.merge(res_le, res_gdp)

4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [17]:
endpoint_count = 'https://api.worldbank.org/V2/country/all?format=json'
params = {
    'per_page': 20000,
}
response_count = requests.get(endpoint_count, params = params)

In [18]:
res_count = response_count.json()

In [19]:
res_count = pd.json_normalize(res_count[1]).drop(columns=['longitude', 'latitude', 'region.id', 'region.iso2code', 'region.value', 'adminregion.id', 'adminregion.iso2code', 'adminregion.value']).rename(columns={'name':'country'})

In [20]:
res_count

,id,iso2Code,country,capitalCity,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value
0,ABW,AW,Aruba,Oranjestad,HIC,XD,High income,LNX,XX,Not classified
1,AFE,ZH,Africa Eastern and Southern,,NA,NA,Aggregates,,,Aggregates
2,AFG,AF,Afghanistan,Kabul,LIC,XM,Low income,IDX,XI,IDA
3,AFR,A9,Africa,,NA,NA,Aggregates,,,Aggregates
4,AFW,ZI,Africa Western and Central,,NA,NA,Aggregates,,,Aggregates
...,...,...,...,...,...,...,...,...,...,...
291,XZN,A5,Sub-Saharan Africa excluding South Africa and ...,,NA,NA,Aggregates,,,Aggregates
292,YEM,YE,"Yemen, Rep.",Sana'a,LIC,XM,Low income,IDX,XI,IDA
293,ZAF,ZA,South Africa,Pretoria,UMC,XT,Upper middle income,IBD,XF,IBRD
294,ZMB,ZM,Zambia,Lusaka,LMC,XN,Lower middle income,IDX,XI,IDA


In [21]:
pd.merge(res_count, le_gdp)

,id,iso2Code,country,capitalCity,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value,date,life_expectancy,Country,GDP_Per_Capita
0,ABW,AW,Aruba,Oranjestad,HIC,XD,High income,LNX,XX,Not classified,2023,NaN,Africa Eastern and Southern,NaN
1,ABW,AW,Aruba,Oranjestad,HIC,XD,High income,LNX,XX,Not classified,2022,62.899031,Africa Eastern and Southern,42260.451837
2,ABW,AW,Aruba,Oranjestad,HIC,XD,High income,LNX,XX,Not classified,2021,62.454590,Africa Eastern and Southern,38226.146157
3,ABW,AW,Aruba,Oranjestad,HIC,XD,High income,LNX,XX,Not classified,2020,63.313860,Africa Eastern and Southern,29935.070278
4,ABW,AW,Aruba,Oranjestad,HIC,XD,High income,LNX,XX,Not classified,2019,63.755678,Africa Eastern and Southern,39432.128933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4494331,ZWE,ZW,Zimbabwe,Harare,LMC,XN,Lower middle income,IDB,XH,Blend,1964,54.994000,Zimbabwe,NaN
4494332,ZWE,ZW,Zimbabwe,Harare,LMC,XN,Lower middle income,IDB,XH,Blend,1963,54.549000,Zimbabwe,NaN
4494333,ZWE,ZW,Zimbabwe,Harare,LMC,XN,Lower middle income,IDB,XH,Blend,1962,54.071000,Zimbabwe,NaN
4494334,ZWE,ZW,Zimbabwe,Harare,LMC,XN,Lower middle income,IDB,XH,Blend,1961,53.619000,Zimbabwe,NaN
